# Ciencia de Datos Aplicada - Curso Capstone Proyecto Final

# Agrupación y segmentación de Vecindarios en la ciudad de Toronto, Canada

Use el Cuaderno para crear el código para rastrear la siguiente página de Wikipedia, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, para obtener los datos que están en la tabla de códigos postales y transformar los datos en un marco de datos de pandas:

1. Empiece por crear un nuevo Notebook para esta tarea.


In [7]:
# importamos las librerías necesarias
import requests # librería para manejar las solicitudes
import pandas as pd # librería para análisis de datos
import numpy as np # librería para manejar datos vectorizados
import random # librería para generar números aleatorios
from bs4 import BeautifulSoup

!pip install geopy
from geopy.geocoders import Nominatim # módulo para convertir una dirección en valores de latitud y longitud 

# librerías para mostrar imágenes 
from IPython.display import Image 
from IPython.core.display import HTML 
    
# librería para convertir un archivo json en un dataframe pandas
from pandas.io.json import json_normalize


! pip install folium==0.5.0
import folium # librería para graficar 

print('Folium installed')
print('Libraries imported.')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.2/79.2 kB 15.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for folium: filename=folium-0.5.0-py3-none-any.whl size=76117 sha256=c65434da5e675ae4e2cdcc63e3bea5187f6e170c9809897ea0ad0a41f8dcfee1
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/26/e2/90/e29b0a163b7ca7737dd33fc1ad742b38ff55963d8220013134
Successfully built folium
  Attempting uninstall: folium
    Found existing installation: folium 0.11.0
    Uninstalling folium-0.11.0:
      Successfully uninstalled folium-0.11.0
Folium installed
Libraries imported.


Por medio de la librería BeautifulSoup se hace Web Scraping para obtener los datos de la página web que contiene los códigos postales de Toronto.

In [9]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(URL)
soup = BeautifulSoup(page.content, "html.parser")
table=soup.table

Obtenemos el texto de las celdas que contienen los datos de los municipios.

In [10]:
file=[]
ncol=list(range(0,9))

for i in ncol :
    
    for row in table.find_all('tr'): # in html table row is represented by the table
    # Get all columns in each row.
        cols = row.find_all('p') # in html a column is represented by the tag td or p
        file.append(cols[i].getText())

    # convert to dataframe:
df= pd.DataFrame(file)
df.head()

,0
0,M1ANot assigned\n
1,M1BScarborough(Malvern / Rouge)\n
2,M1CScarborough(Rouge Hill / Port Union / Highl...
3,M1EScarborough(Guildwood / Morningside / West ...
4,M1GScarborough(Woburn)\n


Use el Cuaderno para crear el código para rastrear la siguiente página de Wikipedia, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, para obtener los datos que están en la tabla de códigos postales y transformar los datos en un marco de datos de pandas como el que se muestra a continuación:

In [11]:
df["Postal Code"]=df[0].str[0:3]
df["Borough"]=df[0].str[3:].str.split("(",n=1,expand=True)[0]
df["Neigh"]=df[0].str.rsplit("(",n=2,expand=True)[1].str.split(")",n=1,expand=True)[0]
df["Neighbourhood"]=df["Neigh"].str.replace(" / ",",")
postalcode_list=df[(df["Borough"]!="Not assigned\n")]
postalcode_list=postalcode_list.drop([0,"Neigh"],axis=1)
postalcode_list.head()

,Postal Code,Borough,Neighbourhood
1,M1B,Scarborough,"Malvern,Rouge"
2,M1C,Scarborough,"Rouge Hill,Port Union,Highland Creek"
3,M1E,Scarborough,"Guildwood,Morningside,West Hill"
4,M1G,Scarborough,Woburn
5,M1H,Scarborough,Cedarbrae


In [12]:
postalcode_list.shape

(103, 3)

# Importando Coordenadas de los Municipios

En caso de que no pueda obtener las coordenadas geográficas de los vecindarios usando el paquete Geocoder, aquí hay un enlace a un archivo csv que tiene las coordenadas geográficas de cada código postal: http://cocl.us/Geospatial_data
Utilice el paquete Geocoder o el archivo csv para crear el siguiente marco de datos:

In [14]:
#"http://cocl.us/Geospatial_data"
req = requests.get('http://cocl.us/Geospatial_data')
url_content = req.content
csv_file = open('Geospatial_Coordinates.csv','wb')
csv_file.write(url_content)
Geospatial_Coordinates=pd.read_csv("Geospatial_Coordinates.csv")
Geospatial_Coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Agregamos las coordenadas a nuestro dataframe

In [15]:
coord_list=pd.merge(postalcode_list,Geospatial_Coordinates,on="Postal Code")
coord_list.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill,Port Union,Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Agrupación y Segmentación

In [16]:
#Utilizando la libreria geopy, se obtiene las coordenadas de Toronto
address = 'TORONTO,Ontario'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


Seleccionamos vecindarios que tengan la palabra toronto

In [17]:
toronto_data = coord_list[coord_list['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data.groupby("Borough").size().sort_values(ascending=False)

Borough
Downtown Toronto                                                17
Central Toronto                                                  9
West Toronto                                                     6
East Toronto                                                     4
Downtown TorontoStn A PO Boxes25 The Esplanade                   1
East TorontoBusiness reply mail Processing Centre969 Eastern     1
East YorkEast Toronto                                            1
dtype: int64

Creamos un mapa con Toronto, y añadimos los marcadores

In [18]:
# crear un mapa de Manhattan usando los valores de latitud y longitud
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# añadir los marcadores al mapa
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto